<a href="https://colab.research.google.com/github/alvumu/BDGE/blob/main/ClickHouse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ClickHouse**

ClickHouse representa un sistema de gestión de bases de datos (DBMS) de código abierto, centrado en columnas y diseñado específicamente para el procesamiento analítico en línea (OLAP). Esta herramienta innovadora fue desarrollada por Yandex y actualmente impulsa la segunda plataforma de análisis web más extensa, Yandex Metrica. Destacando como el primer almacén de datos SQL de código abierto, ClickHouse ha logrado igualar y superar la escalabilidad y rendimiento de bases de datos reconocidas como Veryica y Snowflake. Su enfoque columnar, combinado con una eficiente gestión de datos, lo posiciona como una opción destacada para abordar desafíos analíticos en entornos que demandan respuestas rápidas y procesamiento eficaz de grandes volúmenes de datos.




# Características

Algunas de las características que ofrece este sistema de gestión de bases de datos son los siguientes :  

* Una base de datos orientada a **columna**, sólo almacena los datos.
* Los **datos** se comprimen reduciendo el espacio y se **almacenan** en **disco**
* Capaz de **procesar** en **paralelo** los datos exprimiendo las capacidades multicores de los servidores.
* Destaca por procesar datos mediante vectores, **optimizando** la **eficiencia** de la CPU
* Utiliza el **lenguaje SQL** para ejecución de las sentencias.
* Permite **actualizar** los **datos en tiempo real** sin bloqueo.
* Proporciona **indexación** por **clave primaria**. Hace posible recuperar la información almacenada para un determinado valor o un rango de valores con muy baja latencia y pocos milisegundos.
* Puede trabajar con millones de registros en bruto (sin realizar preprocesamientoo previo de los datos) consiguiendo ejecutar **sentencias sin ninguna latencia**
* Soporte de **replicación** e **integridad** de datos.



# Instalación

A continuación, se procede a la instalación del software de ClickHouse en un entorno Python, con el objetivo de habilitar su uso en Google Colab. Se proporcionarán instrucciones detalladas sobre los pasos necesarios para lograr una integración exitosa de ClickHouse con Python.

In [10]:
pip install clickhouse-connect

In [11]:
import clickhouse_connect

client = clickhouse_connect.get_client(host='fgi402x8e7.europe-west4.gcp.clickhouse.cloud', port=8443, username='default', password='SY_rjdkJKk7oU')

In [12]:
import gzip
from urllib.request import Request,urlopen
import io
import os
import os.path as path

def download_csv(baseurl, filename):
    file = path.abspath(path.join(os.getcwd(),filename))
    request = Request(baseurl + filename + '.gz')
    response = urlopen(request)
    buf = io.BytesIO(response.read())
    f = gzip.GzipFile(fileobj=buf)
    data = f.read()
    with open (filename, 'wb') as ff:
      ff.write(data)

baseurl = 'https://raw.githubusercontent.com/dsevilla/bdge-data/master/es.stackoverflow/'
download_csv(baseurl, 'Posts.csv')
download_csv(baseurl, 'Users.csv')
download_csv(baseurl, 'Tags.csv')
download_csv(baseurl, 'Comments.csv')
download_csv(baseurl, 'Votes.csv')

In [65]:
!cp /content/Posts.csv /var/lib/clickhouse/user_files/Posts.csv

cp: cannot create regular file '/var/lib/clickhouse/user_files/Posts.csv': No such file or directory


In [13]:
client.query("CREATE DATABASE IF NOT EXISTS stackoverflow")

In [75]:
client.query("DROP TABLE IF EXISTS stackoverflow.PostsPart")
client.query("""
CREATE TABLE IF NOT EXISTS stackoverflow.PostsPart
(
    Id Int32,
    AcceptedAnswerId Nullable(Int32),
    AnswerCount Nullable(Int32),
    Body String,
    ClosedDate Nullable(DateTime),
    CommentCount Nullable(Int32),
    CommunityOwnedDate Nullable(DateTime),
    CreationDate DateTime,
    FavoriteCount Nullable(Int32),
    LastActivityDate Nullable(DateTime),
    LastEditDate Nullable(DateTime),
    LastEditorDisplayName String,
    LastEditorUserId Nullable(Int32),
    OwnerDisplayName String,
    OwnerUserId Nullable(Int32),
    ParentId Nullable(Int32),
    PostTypeId UInt8,
    Score Int32,
    Tags String,
    Title String,
    ViewCount Nullable(Int32)
) ENGINE = MergeTree
 PRIMARY KEY Id
 ORDER BY Id;""")

In [81]:
client.query("INSERT INTO stackoverflow.PostsPart FORMAT CSVWithNames")

NameError: name 'Posts' is not defined

Referencias

https://vasexperts.com/es/resources/glossary/clickhouse/ (ClickHouse)

https://www.adictosaltrabajo.com/2019/02/18/introduccion-a-clickhouse/#01 (Caracteristicas)

https://clickhouse.com/docs/en/integrations/python (Instalación)


https://www.percona.com/blog/clickhouse-new-opensource-columnar-database/

https://www.linkedin.com/pulse/clickhouse-un-motor-de-base-sql-super-rápid-ezequiel-paolillo/?originalSubdomain=es
